<img src="./img/HWNI_logo.svg"/>

# Lab B - Paired t-tests and Non-Parametric Tests - Solutions

In [ ]:
# makes our plots show up inside Jupyter
%matplotlib inline

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import scipy.stats

import util.utils as utils
import util.shared as shared

shared.format_plots()
shared.format_dataframes()

In this lab, we'll look at some more commonly-used and practical hypothesis tests than the simple example used in the
[hypothesis testing lab](../03 - Hypothesis Testing/Lab - Hypothesis Testing.ipynb).
In these tests, instead of collecting only one data point in our experimental condition, we'll collect multiple data points. All the tests considered in this lab -- paired and unpaired t-tests, non-parametric tests -- arise due to different choices of test statistic.

In this half of the lab, we'll be looking at using
[paired t-tests](https://en.wikipedia.org/wiki/Student%27s_t-test)
and at two non-parametric tests:
one is the randomization test described in the
[tutorial](Tutorial - Tests for 2-Sample Data.ipynb)
and the other is called the
[*Wilcoxon signed-rank test*](https://en.wikipedia.org/wiki/Wilcoxon_signed-rank_test)

## Dataset Introduction

Certain types of nerve cells have the ability to regenerate a part of the cell that has been amputated. In an early study of this process, measurements were made on the nerves in the spinal cord of rhesus monkeys. Nerves emanating from the left side of the cord were cut, while nerves from the right side were kept intact. During the regeneration process, the amount of creatine phosphate (CP) was measured in the left and right portions of the spinal cord. You are interested in whether CP levels are different between the cut and control sides of the spinal cord. 

 *adapted from Samuels & Witmer, pg 387. Originally: Bodian (1947)*

**Q1** Why is a paired test appropriate for this dataset? What are we assuming about the data when we choose to use a paired t-test in particular?

**Q2** This experiment could've been run by severing both sides of the spinal cord in one set of monkeys and neither side in a control group. What makes the original experimental design likely superior?

## Loading and Tidying Data

We begin by loading the data and taking a look at it. As is common in studies involving primates, the number of organisms used is quite small, so we can view all of the data at once.

In [ ]:
cp_data = pd.read_csv("./data/3b.csv",index_col=None)

In [ ]:
cp_data

Notice, however, that this time our data is not "tidy" -- our "unit of observation" is one organism, but each row doesn't correspond to an organism. Instead, it corresponds to a measurement of creatine phosphate levels.

Luckily, our collaborator has included enough information that we can tidy this data up. The dataframe that we want to have has rows corresponding to values of `organismIdx` and two new columns, `Control_R` and `Regen_L`, which should be populated with values from `cpLevel`.

This operation of "reorganization" is common enough that pandas has a function to do it for us: `dataframe.pivot`. We tell is which column gives us the row `index`, which column gives us the new `columns`, and which column gives us the new `values`, like so:

In [ ]:
tidy = cp_data.pivot(index="organismIdx", columns="side", values="cpLevel")
tidy

Because the first format is always longer than the second, it's also called the *long format*, while the tidy format is also called the "wide format". As more and more folks get on board the tidy data train, these non-judgmental names for the formats are likely to fall out of favor.

For more on the philosophy of tidy data (written from an R perspective), check out this [paper by Hadley Wickham](http://www.jeannicholashould.com/tidy-data-in-python.html), developer of some of the most prominent stats packages for the R programming language. For some more practical examples of cleaning up messy data in Python, check out [this blog post by Jean Nicholas Hould](http://www.jeannicholashould.com/tidy-data-in-python.html).

## Visualizing Paired Data

If you need more convincing that tidy data formats are better, check out the code cell below. In just one line, it produces a histogram for each column and a scatterplot for each pair of columns -- what seaborn calls a `pairplot`. These can be very useful for exploring lots of pairwise relationships by eye very quickly.

In [ ]:
sns.pairplot(tidy, size=4, aspect=1);

As another, simpler, way to visualize the data, compute the differences between the control and regenerating sides and make a stripplot.

** Q3 ** Why do we take the difference? What does this have to do with our choice of the paired t-test?

In [ ]:
# your code here

** Q4 **Based on this visualization, do you expect the result to be statistically significant or not? Why or why not?

More common than either of these visualizations, at least in the biological literature, is a plot that shows the observations as neighboring stripplots with lines connecting pairs.

Run the cell below to generate just such a plot. Check out the `utils` module if you want to see how it's done -- since seaborn doesn't offer this as a basic plot, it requires a bit of matplotlib-fu.

In [ ]:
utils.difference_plot(tidy, ylabel="Creatine Phosphate Levels")

** Q5 ** What visual aspects of this plot indicate whether an effect is likely to be statistically significant or insignificant? Of the plots we made -- the pairplot scatter plots, the stripplot of differences, or the difference plot -- which do you prefer? Why? Which would you use in a publication?

## Running a Paired t-Test

First, run the unpaired version of the t-test, as in the first half of the lab.

** Q6 ** Is the result significant at $\alpha$ = 0.05? (NB: Future uses of the term "significant" imply significant at this level of $\alpha$).

In [ ]:
# your code here

Scipy also has a paired version of the t-test: `scipy.stats.ttest_rel`, for `rel`ated measurements `t-test`. Run the paired test (the syntax for the function is much the same as for the unpaired test). 

** Q7 ** Is the result significant? If you got different answers for the two tests, explain why this is the case.

In [ ]:
# your code here

** Q8 **Say I gave you one measurement of creatine phosphate from a new organism's spinal cord. Based on your results, would you expect to be able to tell whether this measurement was from a spine undergoing regeneration? Why or why not? Does your answer change if I give you more measurements, all from different organisms in the same group?

** Q9 ** Now say I gave you two measurements of creatine phosphate, both from the same organism, one before and one after severing its spinal cord. Based on your results, do you expect to be able to tell which is which? Why or why not?

## Non-Parametric Test: Wilcoxon

** Q9 ** What would be the argument for using a non-parametric test in this case? Are there any arguments against?

Use the wilcoxon signed-rank test to perform a non-parametric paired test. Unfortunately, scipy doesn't implement an exact version of the signed-rank test for small sample sizes, so the p-value you calculate will be incorrect and you should receive a `"Warning: sample size too small for normal approximation"`.

** Q10 ** Despite this error, and the weaker power of the non-parametric test, do you report a significant difference based on the Wilcoxon test?

In [ ]:
scipy.stats.wilcoxon(tidy.Control_R, tidy.Regen_L)

## Non-Parametric Test: Randomization

The
[tutorial](Tutorial - Tests for 2-Sample Data.ipynb)
describes the motivation for *randomization tests*,
a general class of non-parametric tests that can be used with a wide variety of test statistics,
including cases where parametric tests like the $t$-test fail.

Unfortunately, randomization tests are not implemented in the major Python statistical libraries.
One reason why is that it's pretty easy to write your own!
You just need to write

- a randomization, or shuffling, method appropriate to your null hypothesis
- a function that computes your test statistic
- a loop that will randomly shuffle your data and then compute the test statistic
- a function to calculate the $p$-value

The `util` module contains some code to shuffle dataframes.

The function to compute the test statistic appears in the cell below.
Notice that we can just change the statistic calculated by this function --
from `mean` to `median` or `max` or whatever we want --
and we'll get a new statistical test.
Much easier than implementing the Wilcoxon test AND the $t$-test!

In [ ]:
def compute_test_statistic(tidy_dataframe, statistic='mean'):
    
    differences = tidy_dataframe.iloc[:,0] - tidy_dataframe.iloc[:,1]
    
    if statistic == 'mean':
        return np.mean(differences)
    elif statistic == 'median':
        return np.median(differences)
    elif statistic == 'max':
        return np.max(differences)

In [ ]:
observed_test_statistic = compute_test_statistic(tidy, statistic = 'mean')

The loop to draw from the sampling distribution of the test statistic
under the null distribution appears in the cell below.
We just repeatedly shuffle the original data frame and
compute the test statistic,
storing the results in the array
`null_test_statistics`.

In [ ]:
num_shuffles = 1000 # how many samples do we draw from the null distribution?

null_test_statistics = np.zeros(num_shuffles) # pre-allocate some memory to store the results of these samples

for shuffle_idx in range(num_shuffles):
    #first we shuffle the dataframe
    shuffled_df = utils.make_shuffle(tidy)
    #then we compute the test statistic and add it to our array
    null_test_statistics[shuffle_idx] = compute_test_statistic(shuffled_df, statistic = 'mean')

Lastly, we need to calculate the $p$-value.
For a one-tailed test, this is super easy!
We just need to calculate the fraction of times
that the test statistic after shuffling was at least as high
as it was before shuffling.

We can do this in one line of Python:
first, we get a Boolean array
(an array of `True`s and `False`s)
using the `>=` operator.
This array has a `True` wherever the value in the 
`null_test_statistic` array
was at least as large as the
`observed_test_statistic`.

We then calculate the mean of that array using
`np.mean`.
Since `True` is treated as a `1`
and `False` is treated as a `0`,
taking the average value gives us the fraction
of values that were `True` --
the fraction of times the test statistic calculated from a shuffled data frame
was higher than the one we calculated from our data.

In [ ]:
np.mean(null_test_statistics >= observed_test_statistic)

#### Q11 Is the result significant according to the randomization test?

#### Q12 Change the argument `statistic` to `median` in both cells above. Is the result significant?

#### Q13 Explain the benefits and drawbacks of the randomization test relative to a) parametric methods and b) test-statistic-specific non-parametric methods like the Wilcoxon test.